In [1]:
import numpy as np
import pandas as pd

from func import (compute_pi_nsw, 
                  compute_pi_nsw_lag1, 
                  compute_obj_val, 
                  synthesize_rel_mat, 
                  exam_func)

In [2]:
n_query = 20
n_doc = 10
k = 3
default_lam = 1
default_noise = 0.05
flip_ratio = 0.3
random_state = 1234

In [3]:
v = exam_func(n_doc, k, "inv")

rel_mat_true, rel_mat_obs = synthesize_rel_mat(
                n_query=n_query,
                n_doc=n_doc,
                lam=default_lam,
                noise=default_noise,
                flip_ratio=flip_ratio,
                random_state=random_state,
            )

In [6]:
pi_0 = compute_pi_nsw(rel_mat=rel_mat_obs, v=v)
pi_1 = compute_pi_nsw_lag1(rel_mat=rel_mat_obs, v=v, max_iter=5, verbose=True)

opt_val_0 = compute_obj_val(pi_0, rel_mat_obs, v)
opt_val_1 = compute_obj_val(pi_1, rel_mat_obs, v)

100%|██████████| 5/5 [00:02<00:00,  1.85it/s]


In [5]:
result_df = pd.DataFrame([
    [pi_0.sum(axis=2).max(), pi_1.sum(axis=2).max()],
    [pi_0.sum(axis=1).max(), pi_1.sum(axis=2).max()],
    [opt_val_0, opt_val_1]
    ], 
    index = [
        "Max Sum pi over k",
        "Max sum pi over i",
        "Optimal Value"
    ],
    columns = [
        "Original NSW", "Lagrangified NSW"
    ]
)

print(result_df)
print("-------------------------------------------------------")
print("Maximum Difference of pi:", np.abs(pi_0 - pi_1).max())



                   Original NSW  Lagrangified NSW
Max Sum pi over k      1.000007          1.000005
Max sum pi over i      1.000019          1.000005
Optimal Value          8.377851          8.377838
-------------------------------------------------------
Maximum Difference of pi: 0.07029102677974508


#### **Validation of 1st Constraint**
The first constraint of original NSW states that the sum of pi over k must be less than or equalt to 1. According to the result of Lagrangified NSW, we can see that the maximum of sum of pi over k is 1. This means that our attempt to lagrangify the first constraint works as expected. 

#### **Validation of 2nd Constraint**
The second constraint of original NSW states that the sum of pi over i must be less than or equalt to 1. According to the result of Lagrangified NSW, we can see that the maximum of sum of pi over i is 1. This is expected since we keep the second constraint as is. 

#### **Optimal Value**
The optimal values of both Original NSW and Lagrangified NSW are relatively the same and thus further validate the result of Lagrangified NSW

#### **Maximum Difference of pi**
Since the goal of this project is to compare the outcome envy from Original NSW and Lagrangified NSW, we must check how close are the pi from both algorithms. If both pi are almost identical, then there's no point in measuring the difference envy since the outcome envy must also be relatively the same. According to the result, we can see that the maximum difference of pi between the 2 algorithm is 0.07 (can be varied depending on the experiment parameters), which is 7% of the maximum possible difference. Thus, the difference is significant and it is worth exploring the difference of envy.